This code was based on in the following references:

[1] "Particle Swarm Optimization" published in 1995 by James Kennedy and Russell Eberhart

[2] "A New Optimizer Using Particle Swarm Theory" published in 1995 by James Kennedy and Russell Eberhart

[3] "A Novel Search Algorithm based on Fish School Behavior" published in 2008 by Bastos Filho, Lima Neto, Lins, D. O. Nascimento and P. Lima

[4] "An Enhanced Fish School Search Algorithm" published in 2013 by Bastos Filho and  D. O. Nascimento

[5] "Defining a Standard for Particle Swarm Optimization" published in 2007 by Bratton and Kennedy

### Imports

In [1]:
import os
import sys
import csv
import random
import copy
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
from google.colab import drive, files
%matplotlib inline
plt.rcParams["figure.figsize"] = (13,8)

### Connection with Google Drive

In [2]:
drive.mount('/content/drive', force_remount=True) #  Connect to drive
FOLDERNAME = 'POLI/decimo/tcc/Algoritms/Algoritms Notebook/' #  Set folder that have the datasts
assert FOLDERNAME is not None, "[!] Enter the foldername."
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME)) #  Locale the foldeer

%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/POLI/decimo/tcc/Algoritms/Algoritms Notebook


### Class of Particles and PSO with each method to execute optimization

In [3]:
from ObjectiveFunction import *
from SearchSpaceInitializer import UniformSSInitializer, OneQuarterDimWiseSSInitializer
%cd ../

class Particle(object):
  """
  Classe que define um particula que contém sua posição (em diversas dimensões)
  com o custo de desempenho. Para auxilio também memoriza o quanto andou
  e quanto de custo mudou.
  """
  def __init__(self, dim):
    self.pos = np.array([np.nan for _ in range(dim)])
    self.score = np.inf
    self.vel = np.array([random.uniform(-1,1) for _ in range(dim)])
    self.best_score = np.inf
    self.best_pos = np.array([np.nan for _ in range(dim)])
  
  def update_bests(self, score=None, pos=None):
    score = self.score if score is None else score
    pos = self.pos if pos is None else pos
    if score < self.best_score or self.best_score == np.inf:
      self.best_pos = pos
      self.best_score = score


class PSO(object):
  def __init__(self, objective_function, space_initializer, n_iter, part_size,
               inertia=0.5, cognitive=1, social=2, vel_max=10, topology='GBEST',
               clerck=True):
    self.objective_function = objective_function # função de avalição de custo
    self.space_initializer = space_initializer # posições iniciais dos peixes

    self.dim = objective_function.dim
    self.minf = objective_function.minf # limite minimo da função
    self.maxf = objective_function.maxf # limite máximo da função
    self.n_iter = n_iter
    self.topology = topology
    self.clerck = clerck

    self.part_size = part_size  # quantidade de peixes
    self.w = inertia
    self.c1 = cognitive
    self.c2 = social
    self.v_max = vel_max
    self.g_best = None
    
    self.optimum_score_tracking_iter = []
    self.optimum_score_tracking_eval = []
  
  def __init_score_tracking(self):
    self.optimum_score_tracking_iter = []
    self.optimum_score_tracking_eval = []

  def __init_particle(self, pos):
    particle = Particle(self.dim)
    particle.pos = pos
    particle.score = self.objective_function.evaluate(particle.pos)
    # particle.update_bests(pos=pos)
    self.optimum_score_tracking_eval.append(self.g_best.score)
    return particle

  def __init_pso(self):
    self.g_best = Particle(self.dim)
    self.particles = []
    
    positions = self.space_initializer.sample(self.objective_function, self.part_size)

    for idx in range(self.part_size):
      particle = self.__init_particle(positions[idx])
      self.particles.append(particle)
    self.update_bests_particle()
    self.optimum_score_tracking_iter.append(self.g_best.score)

  def update_bests_particle(self):
    for i, p in enumerate(self.particles):
      p.update_bests()
      if p.score < self.g_best.score or self.g_best.score == np.inf:
        self.g_best = copy.copy(p)


  def get_lbest(self, particleA, particleB):
    #if betterthan(particleA.fitness, particleB.fitness, maximization=True):
    return particleA if particleA.score <= particleB.score else particleB


  def constriction_factor(self, iteration):
    """It was found that when rho < 4, the swarm would slowly
    "spiral" toward and around the best found solution in the
    search space with no guarantee of convergence, while for
    rho > 4 convergence would be quick and guaranteed. (Defining a Standard
    for Particle Swarm Optimization)"""

    rho = self.c1 + self.c2
    if rho < 4:
      print(f"rho = {rho}")

    return (2 / abs(2 - rho - math.sqrt(rho**2 - 4 * rho)))

  def collective_movement(self):
    for i, p in enumerate(self.particles):
      if self.topology == "LBEST": # best of neighbors
        best = self.get_lbest(self.particles[(i - 1) % len(self.particles)], self.particles[(i + 1) % len(self.particles)])
      else: # best of all
        best = self.g_best
  
      new_pos = np.zeros((self.dim,), dtype=float)
      for dim in range(self.dim):

        if self.clerck:
          w = 1
          factor = self.constriction_factor(i)
        else:
          w = self.w
          factor = 1

        r1 = random.random()
        r2 = random.random()
        vel_cognitive = self.c1 * r1 * (p.best_pos[dim] - p.pos[dim])
        vel_social = self.c2 * r2 * (best.pos[dim] - p.pos[dim])
        p.vel[dim] = factor * (w * p.vel[dim] + vel_cognitive + vel_social)
        new_pos[dim] = p.pos[dim] + p.vel[dim]

        if new_pos[dim] < self.minf:
          new_pos[dim] = self.minf
        elif new_pos[dim] > self.maxf:
          new_pos[dim] = self.maxf
      self.optimum_score_tracking_eval.append(self.g_best.score)
      score = self.objective_function.evaluate(new_pos)
      p.pos = new_pos
      p.score = score

  def optimize(self):
    self.__init_score_tracking()
    self.__init_pso()

    for i in range(self.n_iter):
      self.collective_movement()
      self.update_bests_particle()
      self.optimum_score_tracking_iter.append(self.g_best.score)

/content/drive/My Drive/POLI/decimo/tcc/Algoritms


### Validate PSO

Validate learning in each iteration

In [4]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for d in [15, 30, 50, 100]:
    print (f"starting VGWO ({d})")
    search_space_initializer = UniformSSInitializer()
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + f"{d}d" + os.sep
    num_exec = 1
    part_size = 30
    num_iterations = 1000
    inertia = 0
    cognitive = 2

    dim = d

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction]
    regular_functions = unimodal_funcs + multimodal_funcs

    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + "/PSO_iter.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['opt', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_iterations)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(dim)
      start = time.time()
      bests_iter, bests_eval = run_experiments(num_iterations, part_size, func, search_space_initializer)
      end = time.time()
      row_csv = ['PSO', func.function_name, (end - start)] + [(b if b != np.inf else 9.9e+999) for b in bests_iter[:num_iterations]]
      writer_csv.writerow(row_csv)
      print(func.function_name, end - start, bests_iter[-1])
      
    f_handle_csv.close()


def run_experiments(n_iter, part_size, objective_function, search_space_initializer):
  opt1 = PSO(objective_function=objective_function, space_initializer=search_space_initializer,
              n_iter=n_iter, part_size=part_size, inertia=0.73, cognitive=2.05,
             social=2.05, vel_max=10, topology='GBEST',clerck=True)
  opt1.optimize()
  return opt1.optimum_score_tracking_iter, opt1.optimum_score_tracking_eval

main()

starting VGWO (15)
Sphere 1.8808610439300537 8.705030632926733e-39
RotatedHyperEllipsoid 4.685927391052246 5.55587283211424e-34
Rosenbrock 2.2536792755126953 0.07847950146881887
Dixon-Price 2.3213064670562744 4.095370433113619e-20
Quartic-Noise 2.0563387870788574 0.008804439607212515
Generalized-Shwefel 1.9072818756103516 0.00019091349349764641
Rastrigin 2.0703468322753906 18.90420697131532
Ackley 2.338833808898926 7.549516567451064e-15
Griewank 3.140643835067749 0.06394362588067715
Levi 3.972933292388916 2.473805829158244
starting VGWO (30)
Sphere 3.388903856277466 1.2950801186664224e-16
RotatedHyperEllipsoid 9.05434274673462 5.519820100759903e-12
Rosenbrock 3.9137959480285645 3.9648247705751953
Dixon-Price 4.5450074672698975 0.005260805260843912
Quartic-Noise 3.888587713241577 8.060234223260167
Generalized-Shwefel 3.5491902828216553 2941.3696888445884
Rastrigin 3.968991994857788 124.5113095117286
Ackley 3.8733773231506348 3.0257790359335215
Griewank 5.27006459236145 0.012320988885877

Validate bests results in multiple sessions of iterations

In [5]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for d in [15, 30, 50, 100]:
    print (f"starting VGWO ({d})")
    search_space_initializer = UniformSSInitializer()
    file_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Executions" + os.sep + f"{d}d" + os.sep
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + f"{d}d" + os.sep
    num_exec = 30
    part_size = 30
    num_iterations = 1000
    inertia = 0
    cognitive = 2

    dim = d

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction]
    regular_functions = unimodal_funcs + multimodal_funcs

    # regular_functions = [RastriginFunction, RosenbrockFunction]

    # Notice that for CEC Functions only the following dimensions are available:
    # 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100
    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + "/PSO_exec.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['opt', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_exec)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(dim)
      
      start = time.time()
      runs = run_experiments(num_iterations, part_size, num_exec, func,
                            search_space_initializer, file_path)
      end = time.time()
      row_csv = ['PSO', func.function_name, (end - start)] + [r for r in runs]
      writer_csv.writerow(row_csv)
      
    f_handle_csv.close()


def run_experiments(n_iter, part_size, num_runs, objective_function,
                    search_space_initializer, save_dir):
  alg_name = "PSO"
  console_out = "Algorithm: {} Function: {} Execution: {} Best Cost: {}"
  if save_dir:
    create_dir(save_dir)
    f_handle_cost_iter = open(save_dir + "/PSO_" + objective_function.function_name + "_cost_iter.txt", 'w+')
    f_handle_cost_eval = open(save_dir + "/PSO_" + objective_function.function_name + "_cost_eval.txt", 'w+')
    runs = []

  for run in range(num_runs):
    opt1 = PSO(objective_function=objective_function, space_initializer=search_space_initializer,
               n_iter=n_iter, part_size=part_size, inertia=0.73, cognitive=2.05,
               social=2.05, vel_max=10, topology='GBEST',clerck=True)
    
    opt1.optimize()
    opt1.g_best.score = opt1.g_best.score if opt1.g_best.score != np.inf else 9.9e+999
    runs.append(opt1.g_best.score)
    print (console_out.format(alg_name, objective_function.function_name, run+1, opt1.g_best.score))

    temp_optimum_cost_tracking_iter = np.asmatrix(opt1.optimum_score_tracking_iter)
    temp_optimum_cost_tracking_eval = np.asmatrix(opt1.optimum_score_tracking_eval)

    if save_dir:
      np.savetxt(f_handle_cost_iter, temp_optimum_cost_tracking_iter, fmt='%.4e')
      np.savetxt(f_handle_cost_eval, temp_optimum_cost_tracking_eval, fmt='%.4e')

  if save_dir:
    f_handle_cost_iter.close()
    f_handle_cost_eval.close()

  return runs

main()

starting VGWO (15)
Algorithm: PSO Function: Sphere Execution: 1 Best Cost: 3.975201534503876e-36
Algorithm: PSO Function: Sphere Execution: 2 Best Cost: 1.223410842544699e-35
Algorithm: PSO Function: Sphere Execution: 3 Best Cost: 6.977715616127229e-35
Algorithm: PSO Function: Sphere Execution: 4 Best Cost: 4.218595100323408e-37
Algorithm: PSO Function: Sphere Execution: 5 Best Cost: 8.029057436044899e-37
Algorithm: PSO Function: Sphere Execution: 6 Best Cost: 6.865441571957335e-37
Algorithm: PSO Function: Sphere Execution: 7 Best Cost: 2.263131621393842e-34
Algorithm: PSO Function: Sphere Execution: 8 Best Cost: 4.016259984634176e-36
Algorithm: PSO Function: Sphere Execution: 9 Best Cost: 3.769971021742982e-35
Algorithm: PSO Function: Sphere Execution: 10 Best Cost: 5.588824684309714e-36
Algorithm: PSO Function: Sphere Execution: 11 Best Cost: 6.258910496514251e-37
Algorithm: PSO Function: Sphere Execution: 12 Best Cost: 1.1516378714024293e-34
Algorithm: PSO Function: Sphere Execution